In [3]:
import time
from datetime import datetime
# Start timer
start_time = time.time()
print(f"Program started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
#################

import pyodbc
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

# --- DB Config ---
db_path = r"C:\Users\mpa1326\Calibration\Data\MPAAnalystDB.mdb"
db_password = "mpasecurity"
conn_str = (
    r"DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};"
    fr"DBQ={db_path};"
    fr"PWD={db_password};"
)

def get_best_fit(base_symbol="Fe", analyte_symbol="Cr", poly_degrees=[1,2]):
    conn = pyodbc.connect(conn_str)

    # --- 1. Get element IDs ---
    elem_df = pd.read_sql("SELECT * FROM setElementInfo", conn)
    base_id = elem_df.loc[elem_df["EleSymbol"] == base_symbol, "ElementID"].iloc[0]
    analyte_id = elem_df.loc[elem_df["EleSymbol"] == analyte_symbol, "ElementID"].iloc[0]

    # --- 2. Get concentration data ---
    conc_q = f"""
    SELECT M.SampleID, M.SampleName, M.SUS, D.ElementID, D.Conc
    FROM calSampleMaster AS M
    INNER JOIN calSampleDetails AS D ON M.SampleID = D.SampleID
    WHERE M.SUS <> 1 AND D.ElementID IN ({base_id}, {analyte_id})
    """
    conc_df = pd.read_sql(conc_q, conn)

    conc_pivot = conc_df.pivot_table(index=["SampleID","SampleName"], 
                                     columns="ElementID", values="Conc").reset_index()
    conc_pivot["CR"] = conc_pivot[analyte_id] / conc_pivot[base_id]

    # --- 3. Get pixel info ---
    pixel_df = pd.read_sql("SELECT * FROM setElementPixelInfo", conn)
    analyte_pixels = pixel_df.loc[pixel_df["ElementID"] == analyte_id, "ElementPixelID"].tolist()
    base_pixels = pixel_df.loc[pixel_df["ElementID"] == base_id, "ElementPixelID"].tolist()

    # --- 4. Get intensities ---
    intens_q = f"""
    SELECT SampleID, ElementPixelID, SparkNo, Intensity
    FROM calData
    WHERE ElementPixelID IN ({",".join(map(str, analyte_pixels + base_pixels))})
    """
    intens_df = pd.read_sql(intens_q, conn)

    conn.close()

    intens_pivot = intens_df.pivot_table(index=["SampleID","SparkNo"], 
                                         columns="ElementPixelID", values="Intensity").reset_index()

    data = pd.merge(intens_pivot, conc_pivot[["SampleID","CR"]], on="SampleID")

    # --- 5. Loop over all pixel combinations and sparks ---
    results = []
    for ap in analyte_pixels:
        for bp in base_pixels:
            if ap in data.columns and bp in data.columns:
                for spark in sorted(data["SparkNo"].unique()):
                    spark_data = data[data["SparkNo"] == spark].copy()
                    spark_data["IR"] = spark_data[ap] / spark_data[bp]

                    X = spark_data[["IR"]].values
                    y = spark_data["CR"].values

                    # Try different polynomial degrees
                    for deg in poly_degrees:
                        model = make_pipeline(PolynomialFeatures(degree=deg), LinearRegression())
                        model.fit(X, y)
                        r2 = model.score(X, y)

                        results.append({
                            "AnalytePixel": ap,
                            "BasePixel": bp,
                            "SparkNo": spark,
                            "Degree": deg,
                            "R2": r2
                        })

    results_df = pd.DataFrame(results).sort_values("R2", ascending=False)
    best = results_df.iloc[0]

    return results_df, best

##############
results_df, best = get_best_fit(base_symbol="Fe", analyte_symbol="Al", poly_degrees=[1,2,3])

print("Top 10 results:")
print(results_df.head(10))

print("\nBest overall fit:")
print(best)
##################
end_time = time.time()
print(f"Program ended at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Total execution time: {end_time - start_time:.2f} seconds")

Program started at: 2025-09-05 17:34:54


C:\Users\mpa1326\AppData\Local\Temp\ipykernel_15440\2987753982.py:28: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  elem_df = pd.read_sql("SELECT * FROM setElementInfo", conn)
C:\Users\mpa1326\AppData\Local\Temp\ipykernel_15440\2987753982.py:39: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  conc_df = pd.read_sql(conc_q, conn)
C:\Users\mpa1326\AppData\Local\Temp\ipykernel_15440\2987753982.py:46: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pixel_df = pd.read_sql("SELECT * FROM setElementPixelInfo", conn)
C:\Users\mpa1326\Ap

Top 10 results:
      AnalytePixel  BasePixel  SparkNo  Degree        R2
4295          5715       5870        1       3  0.998514
4294          5715       5870        1       2  0.998507
1745          5713       5846        3       3  0.998500
1744          5713       5846        3       2  0.998500
1844          5713       5857        3       3  0.998485
4433          5715       5885        2       3  0.998473
4007          5715       5838        1       3  0.998469
4006          5715       5838        1       2  0.998463
1823          5713       5855        2       3  0.998463
1822          5713       5855        2       2  0.998459

Best overall fit:
AnalytePixel    5715.000000
BasePixel       5870.000000
SparkNo            1.000000
Degree             3.000000
R2                 0.998514
Name: 4295, dtype: float64
Program ended at: 2025-09-05 17:35:22
Total execution time: 28.21 seconds
